In [1]:
#welcome! you can just click on run.
#code is runnable

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import pandas as pd
 
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import  train_test_split
 
import tensorflow as tf
from tensorflow.keras import regularizers
import pandas as pd

In [2]:
import pandas as pd
data = pd.read_excel('crude oil dataset/MAIN - Copy (2).xlsx')
train = data
#test.tail()

In [3]:
train_gp = train.sort_values('Date').groupby(['DJIA','GDP', 'Date'], as_index=False)
train_gp = train_gp.agg({'WTI':['mean']})
train_gp.columns = ['DJIA','GDP', 'Date', 'WTI']
train_gp.head()

,Date,WTI
0,2017-05-01,48.48
1,2017-06-01,45.18
2,2017-07-01,46.63
3,2017-08-01,48.04
4,2017-09-01,49.82


In [4]:
def series_to_supervised(data, window=1, lag=1, dropnan=True):
    cols, names = list(), list()
    # Input sequence (t-n, ... t-1)
    for i in range(window, 0, -1):
        cols.append(data.shift(i))
        names += [('%s(t-%d)' % (col, i)) for col in data.columns]
    # Current timestep (t=0)
    cols.append(data)
    names += [('%s(t)' % (col)) for col in data.columns]
    # Target timestep (t=lag)
    cols.append(data.shift(-lag))
    names += [('%s(t+%d)' % (col, lag)) for col in data.columns]
    # Put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # Drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [5]:
window = 2
lag = 8
lag_size = 8
series = series_to_supervised(train_gp.drop('Date', axis=1), window=window, lag=lag)
series.tail()

,WTI(t-2),WTI(t-1),WTI(t),WTI(t+8)
49,61.72,65.17,71.38,91.64
50,65.17,71.38,72.49,108.50
51,71.38,72.49,67.73,101.78
52,72.49,67.73,71.65,109.55
53,67.73,71.65,81.48,114.84


In [6]:
series.dropna

<bound method DataFrame.dropna of     WTI(t-2)  WTI(t-1)  WTI(t)  WTI(t+8)
2      48.48     45.18   46.63     62.73
3      45.18     46.63   48.04     66.25
4      46.63     48.04   49.82     69.98
5      48.04     49.82   51.58     67.87
6      49.82     51.58   56.64     70.98
7      51.58     56.64   57.88     68.06
8      56.64     57.88   63.70     70.23
9      57.88     63.70   62.23     70.75
10     63.70     62.23   62.73     56.96
11     62.23     62.73   66.25     49.52
12     62.73     66.25   69.98     51.38
13     66.25     69.98   67.87     54.95
14     69.98     67.87   70.98     58.15
15     67.87     70.98   68.06     63.86
16     70.98     68.06   70.23     60.83
17     68.06     70.23   70.75     54.66
18     70.23     70.75   56.96     57.35
19     70.75     56.96   49.52     54.81
20     56.96     49.52   51.38     56.95
21     49.52     51.38   54.95     53.96
22     51.38     54.95   58.15     57.03
23     54.95     58.15   63.86     59.88
24     58.15     63.86 

In [7]:
#last_item = 'DJIA(t-%d)' % window
#last_store = 'GDP(t-%d)' % window
#series = series[(series['GDP(t+%d)' % lag] == series[last_store])]
#series = series[(series['DJIA(t+%d)'  % lag] == series[last_item ])]
#series = series[(series['GDP(t)'] == series[last_store])]
#series = series[(series['DJIA(t)'] == series[last_item])]
print(series)

    WTI(t-2)  WTI(t-1)  WTI(t)  WTI(t+8)
2      48.48     45.18   46.63     62.73
3      45.18     46.63   48.04     66.25
4      46.63     48.04   49.82     69.98
5      48.04     49.82   51.58     67.87
6      49.82     51.58   56.64     70.98
7      51.58     56.64   57.88     68.06
8      56.64     57.88   63.70     70.23
9      57.88     63.70   62.23     70.75
10     63.70     62.23   62.73     56.96
11     62.23     62.73   66.25     49.52
12     62.73     66.25   69.98     51.38
13     66.25     69.98   67.87     54.95
14     69.98     67.87   70.98     58.15
15     67.87     70.98   68.06     63.86
16     70.98     68.06   70.23     60.83
17     68.06     70.23   70.75     54.66
18     70.23     70.75   56.96     57.35
19     70.75     56.96   49.52     54.81
20     56.96     49.52   51.38     56.95
21     49.52     51.38   54.95     53.96
22     51.38     54.95   58.15     57.03
23     54.95     58.15   63.86     59.88
24     58.15     63.86   60.83     57.52
25     63.86    

In [8]:
#columns_to_drop = [('%s(t+%d)' % (col, lag)) for col in ['DJIA','GDP']]
#for i in range(window, 0, -1):
#    columns_to_drop += [('%s(t-%d)' % (col, i)) for col in ['DJIA','GDP']]
#series.drop(columns_to_drop, axis=1, inplace=True)
#series.drop(['DJIA(t)','GDP(t)'], axis=1, inplace=True)

In [9]:
print(series)

    WTI(t-2)  WTI(t-1)  WTI(t)  WTI(t+8)
2      48.48     45.18   46.63     62.73
3      45.18     46.63   48.04     66.25
4      46.63     48.04   49.82     69.98
5      48.04     49.82   51.58     67.87
6      49.82     51.58   56.64     70.98
7      51.58     56.64   57.88     68.06
8      56.64     57.88   63.70     70.23
9      57.88     63.70   62.23     70.75
10     63.70     62.23   62.73     56.96
11     62.23     62.73   66.25     49.52
12     62.73     66.25   69.98     51.38
13     66.25     69.98   67.87     54.95
14     69.98     67.87   70.98     58.15
15     67.87     70.98   68.06     63.86
16     70.98     68.06   70.23     60.83
17     68.06     70.23   70.75     54.66
18     70.23     70.75   56.96     57.35
19     70.75     56.96   49.52     54.81
20     56.96     49.52   51.38     56.95
21     49.52     51.38   54.95     53.96
22     51.38     54.95   58.15     57.03
23     54.95     58.15   63.86     59.88
24     58.15     63.86   60.83     57.52
25     63.86    

In [10]:
labels_col = 'WTI(t+%d)' % lag_size
labels = series[labels_col]
series = series.drop(labels_col, axis=1)

X_train, X_valid, Y_train, Y_valid = train_test_split(series, labels.values, test_size=0.25, random_state=0)
#X_train = series[:24]
#Y_train = labels.values[:24]
#X_valid = series[24:]
#Y_valid = labels.values[24:]
print('Train set shape', X_train.shape)
print('Validation set shape', X_valid.shape)
X_train.head()

Train set shape (39, 3)
Validation set shape (13, 3)


,WTI(t-2),WTI(t-1),WTI(t)
31,53.96,57.03,59.88
44,40.94,47.02,52.00
45,47.02,52.00,59.04
9,57.88,63.70,62.23
16,70.98,68.06,70.23


In [11]:
epochs = 200
#batch = 256
lr = 0.005
adam = tf.keras.optimizers.Adam

In [12]:


#model_mlp = tf.keras.Sequential()
#model_mlp.add(Dense(3, activation='relu', input_dim=X_train.shape[1]))
#tf.keras.layers.Dense(3, activation='relu',input_dim=2),
#model_mlp.add(Dense(1))
model = tf.keras.Sequential([
tf.keras.layers.Dense(3, activation='relu',input_dim=X_train.shape[1]),
tf.keras.layers.Dense(1)
])
model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.005))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 3)                 12        
                                                                 
 dense_1 (Dense)             (None, 1)                 4         
                                                                 
Total params: 16
Trainable params: 16
Non-trainable params: 0
_________________________________________________________________


In [13]:
history = model.fit(X_train.values, Y_train, validation_data=(X_valid.values, Y_valid), epochs=epochs, verbose=2)

Epoch 1/200
2/2 - 0s - loss: 28.9657 - val_loss: 16.9964 - 486ms/epoch - 243ms/step
Epoch 2/200
2/2 - 0s - loss: 25.4052 - val_loss: 16.4052 - 19ms/epoch - 10ms/step
Epoch 3/200
2/2 - 0s - loss: 22.2657 - val_loss: 17.1785 - 21ms/epoch - 10ms/step
Epoch 4/200
2/2 - 0s - loss: 20.2197 - val_loss: 18.4472 - 23ms/epoch - 12ms/step
Epoch 5/200
2/2 - 0s - loss: 18.8504 - val_loss: 19.7028 - 22ms/epoch - 11ms/step
Epoch 6/200
2/2 - 0s - loss: 17.9601 - val_loss: 20.8932 - 22ms/epoch - 11ms/step
Epoch 7/200
2/2 - 0s - loss: 17.4731 - val_loss: 21.9915 - 21ms/epoch - 11ms/step
Epoch 8/200
2/2 - 0s - loss: 17.3729 - val_loss: 22.9378 - 21ms/epoch - 10ms/step
Epoch 9/200
2/2 - 0s - loss: 17.3879 - val_loss: 23.6720 - 22ms/epoch - 11ms/step
Epoch 10/200
2/2 - 0s - loss: 17.6949 - val_loss: 24.1231 - 22ms/epoch - 11ms/step
Epoch 11/200
2/2 - 0s - loss: 17.7821 - val_loss: 24.2556 - 20ms/epoch - 10ms/step
Epoch 12/200
2/2 - 0s - loss: 17.8255 - val_loss: 24.2271 - 22ms/epoch - 11ms/step
Epoch 13/20

Epoch 100/200
2/2 - 0s - loss: 16.9455 - val_loss: 21.4100 - 20ms/epoch - 10ms/step
Epoch 101/200
2/2 - 0s - loss: 16.9409 - val_loss: 21.3510 - 20ms/epoch - 10ms/step
Epoch 102/200
2/2 - 0s - loss: 16.9457 - val_loss: 21.3142 - 19ms/epoch - 10ms/step
Epoch 103/200
2/2 - 0s - loss: 16.9432 - val_loss: 21.2714 - 20ms/epoch - 10ms/step
Epoch 104/200
2/2 - 0s - loss: 16.9419 - val_loss: 21.2940 - 20ms/epoch - 10ms/step
Epoch 105/200
2/2 - 0s - loss: 16.9326 - val_loss: 21.3650 - 19ms/epoch - 9ms/step
Epoch 106/200
2/2 - 0s - loss: 16.9211 - val_loss: 21.4443 - 20ms/epoch - 10ms/step
Epoch 107/200
2/2 - 0s - loss: 16.9004 - val_loss: 21.6035 - 19ms/epoch - 9ms/step
Epoch 108/200
2/2 - 0s - loss: 16.8839 - val_loss: 21.8700 - 20ms/epoch - 10ms/step
Epoch 109/200
2/2 - 0s - loss: 16.8812 - val_loss: 22.0800 - 19ms/epoch - 10ms/step
Epoch 110/200
2/2 - 0s - loss: 16.9281 - val_loss: 22.2567 - 19ms/epoch - 9ms/step
Epoch 111/200
2/2 - 0s - loss: 16.9412 - val_loss: 22.3539 - 20ms/epoch - 10ms/

Epoch 198/200
2/2 - 0s - loss: 16.5027 - val_loss: 21.5453 - 20ms/epoch - 10ms/step
Epoch 199/200
2/2 - 0s - loss: 16.5388 - val_loss: 21.8096 - 19ms/epoch - 10ms/step
Epoch 200/200
2/2 - 0s - loss: 16.6381 - val_loss: 21.8189 - 21ms/epoch - 10ms/step


In [14]:
mlp_train_pred = model.predict(X_train.values)
mlp_valid_pred = model.predict(X_valid.values)
print('Train rmse:', np.sqrt(mean_squared_error(Y_train, mlp_train_pred)))
print('Validation rmse:', np.sqrt(mean_squared_error(Y_valid, mlp_valid_pred)))

Train rmse: 19.535512013441704
Validation rmse: 23.51662505167784
